In [8]:
import os

people_list = os.listdir("output")
people_list = list(filter(lambda x: "." not in x, people_list))
people_list

['24071616_AD',
 '24071008_AD',
 '24072023_AD',
 '24071617_AD',
 '24071009_AD',
 '24070901_AD',
 '24071618_AD',
 '24070905_AD',
 '24071619_AD',
 '24072825_AD',
 '24070904_AD',
 '24070907_AD',
 '24071010_AD',
 '24071011_AD',
 '24070902_AD',
 '24072926_AD',
 '24070906_AD',
 '24071513_AD',
 '24071721_AD',
 '24071512_AD',
 '24071615_AD',
 'Test_AD']

In [9]:
import pandas as pd

people_fea = {}
for _p in people_list:
    people_fea[_p] = pd.read_csv(os.path.join(f"output/{_p}", "features.csv"), index_col=0)

In [10]:
whole_fea_list = ["Player1AttentionRatio","Player2AttentionRatio","Player1MinToCircle","Player2MinToCircle","MatchRoundRatio",]
saccade_fea_list = ["SaccadeSpeed_Mean","SaccadeSpeed_Max","SaccadeSpeed_Std","SaccadeAngel_Mean","SaccadeAngel_Max","SaccadeAngel_Std","SaccadeDelay","SaccadeDelayPercent","TrajectoryDTW"]

In [11]:
""" 
video_type: 
    [] => all
    p => pingpang
    w => tennis
    1 => first watch
    2 => mutiple watch
    N => no label
    R_S => random shining
    A_S => all shining
    R_A => random arrow
    A_A => all arrow
""" 
def StatsFeatures(all_people_fea:dict, fea_list:list, video_types:list[str]):
    
    res={}
    for _p in people_list:
        video_list = all_people_fea[_p].index.to_list()
        _label_video_list = []
        for _v_t in video_types:
            if _v_t=="p":
                _v_ids = list(filter(lambda x:x.startswith("p"), all_people_fea[_p].index))
            elif _v_t=="w":
                _v_ids = list(filter(lambda x:x.startswith("w"), all_people_fea[_p].index))
            elif _v_t=="1":
                _v_ids = list(filter(lambda x: not ((x.split("-")[0].endswith("_2")) or (x.split("-")[0].endswith("_3")) or (x.split("-")[0].endswith("_4"))), all_people_fea[_p].index))
            elif _v_t=="2":
                _v_ids = list(filter(lambda x: (x.split("-")[0].endswith("_2")) or (x.split("-")[0].endswith("_3")) or (x.split("-")[0].endswith("_4")), all_people_fea[_p].index))
            elif _v_t=="N_L":
                _v_ids = list(filter(lambda x: not (("_S" in x.split("-")[0]) or ("_A" in x.split("-")[0])), all_people_fea[_p].index))
            elif _v_t=="R_S":
                _label_video_list += list(filter(lambda x: "R_S" in x.split("-")[0], all_people_fea[_p].index))
            elif _v_t=="A_S":
                _label_video_list += list(filter(lambda x: "A_S" in x.split("-")[0], all_people_fea[_p].index))
            elif _v_t=="R_A":
                _label_video_list += list(filter(lambda x: "R_A" in x.split("-")[0], all_people_fea[_p].index))
            elif _v_t=="A_A":
                _label_video_list += list(filter(lambda x: "A_A" in x.split("-")[0], all_people_fea[_p].index))
            else:
                _v_ids = list(all_people_fea[_p].index)
            
            video_list = list(set(video_list).intersection(set(_v_ids)))
        if _label_video_list:
            video_list = list(set(video_list).intersection(set(_label_video_list)))

        temp_df = all_people_fea[_p].loc[video_list, fea_list].dropna()

        mean_val = temp_df.mean()
        mean_val.index = [i+"_Mean" for i in mean_val.index]
        std_val = temp_df.std()
        std_val.index = [i+"_Std" for i in std_val.index]
        max_val = temp_df.max()
        max_val.index = [i+"_Max" for i in max_val.index]
        min_val = temp_df.min()
        min_val.index = [i+"_Min" for i in min_val.index]

        res[_p] = {
            **mean_val.to_dict(),
            **std_val.to_dict(),
            **max_val.to_dict(),
            **min_val.to_dict(),
        }

    return pd.DataFrame(res).T

In [12]:
StatsFeatures(
        all_people_fea=people_fea, 
        fea_list=whole_fea_list, 
        video_types=["p", "1", "N_L"]
    ).mean()

Player1AttentionRatio_Mean    0.216087
Player2AttentionRatio_Mean    0.080800
Player1MinToCircle_Mean       0.048747
Player2MinToCircle_Mean       0.052019
MatchRoundRatio_Mean          0.180373
Player1AttentionRatio_Std     0.114137
Player2AttentionRatio_Std     0.074169
Player1MinToCircle_Std        0.039664
Player2MinToCircle_Std        0.044758
MatchRoundRatio_Std           0.130243
Player1AttentionRatio_Max     0.400267
Player2AttentionRatio_Max     0.231563
Player1MinToCircle_Max        0.114105
Player2MinToCircle_Max        0.127320
MatchRoundRatio_Max           0.406468
Player1AttentionRatio_Min     0.088908
Player2AttentionRatio_Min     0.013141
Player1MinToCircle_Min        0.006100
Player2MinToCircle_Min        0.007754
MatchRoundRatio_Min           0.035261
dtype: float64

In [13]:
StatsFeatures(
        all_people_fea=people_fea, 
        fea_list=whole_fea_list, 
        video_types=["p", "1", "R_S", "A_S"]
    ).mean()

Player1AttentionRatio_Mean    0.216283
Player2AttentionRatio_Mean    0.069184
Player1MinToCircle_Mean       0.050127
Player2MinToCircle_Mean       0.055897
MatchRoundRatio_Mean          0.182093
Player1AttentionRatio_Std     0.123510
Player2AttentionRatio_Std     0.053142
Player1MinToCircle_Std        0.043847
Player2MinToCircle_Std        0.045915
MatchRoundRatio_Std           0.126871
Player1AttentionRatio_Max     0.394046
Player2AttentionRatio_Max     0.157172
Player1MinToCircle_Max        0.126159
Player2MinToCircle_Max        0.131524
MatchRoundRatio_Max           0.384708
Player1AttentionRatio_Min     0.069379
Player2AttentionRatio_Min     0.014343
Player1MinToCircle_Min        0.009557
Player2MinToCircle_Min        0.012703
MatchRoundRatio_Min           0.037109
dtype: float64

In [14]:
StatsFeatures(
        all_people_fea=people_fea, 
        fea_list=whole_fea_list, 
        video_types=["p", "1", "R_A", "A_A"]
    ).mean()

Player1AttentionRatio_Mean    0.203994
Player2AttentionRatio_Mean    0.070227
Player1MinToCircle_Mean       0.055636
Player2MinToCircle_Mean       0.053358
MatchRoundRatio_Mean          0.171666
Player1AttentionRatio_Std     0.091796
Player2AttentionRatio_Std     0.052573
Player1MinToCircle_Std        0.034542
Player2MinToCircle_Std        0.033766
MatchRoundRatio_Std           0.097462
Player1AttentionRatio_Max     0.316904
Player2AttentionRatio_Max     0.138803
Player1MinToCircle_Max        0.096803
Player2MinToCircle_Max        0.096851
MatchRoundRatio_Max           0.283354
Player1AttentionRatio_Min     0.115381
Player2AttentionRatio_Min     0.024748
Player1MinToCircle_Min        0.021781
Player2MinToCircle_Min        0.022266
MatchRoundRatio_Min           0.066797
dtype: float64